In [4]:
# %pip install pandas
# %pip install xlrd
import pandas
import re

In [157]:
excel_df = pandas.read_excel("xls_conformidade_gov_20231003_230704981.xls", sheet_name="Planilha2")

In [91]:
excel_df

,SUBSTÂNCIA,CNPJ,LABORATÓRIO,PRODUTO,APRESENTAÇÃO,CLASSE TERAPÊUTICA,TIPO DE PRODUTO (STATUS DO PRODUTO),PF 0%,DOSAGEM
0,BUTILBROMETO DE ESCOPOLAMINA,61.082.426/0002-07,COSMED INDUSTRIA DE COSMETICOS E MEDICAMENTOS ...,BUSCOPAN,20 MG SOL INJ CT 5 AMP VD TRANS X 1 ML,A3A - ANTIESPASMÓDICOS E ANTICOLINÉRGICOS PUROS,Novo,8.88,a
1,DIPIRONA MONOIDRATADA;BUTILBROMETO DE ESCOPOLA...,61.082.426/0002-07,COSMED INDUSTRIA DE COSMETICOS E MEDICAMENTOS ...,BUSCOPAN COMPOSTO,"(4,0 + 500,0) MG/ML SOL INJ IV/IM CT 3 AMP VD ...",A3D - ASSOCIAÇÕES DE ANTIESPASMÓDICOS COM ANAL...,Novo,12.32,a
2,SULFATO DE MAGNÉSIO HEPTAIDRATADO,05.155.425/0001-93,VASCONCELOS INDUSTRIA FARMACEUTICA E COMERCIO ...,SULFATO DE MAGNÉSIO HEPTAHIDRATADO,10% SOL INJ IV CX 50 FR AMP VD TRANS X 50 ML,K4A2 - SOLUÇÕES ELETROLÍTICAS (>20ML E <100ML),Específico,375.00,a
3,CIPIONATO DE TESTOSTERONA,00.923.140/0001-31,EMS SIGMA PHARMA LTDA,DEPOSTERON,100 MG/ML SOL INJ CX 3 AMP VD AMB X 2 ML,"G3B - ANDRÓGENOS EXCLUINDO G3E, G3F",Similar,159.70,a
4,CLORIDRATO DE ESCETAMINA,51.780.468/0001-87,JANSSEN-CILAG FARMACÊUTICA LTDA,SPRAVATO,"140 MG/ML SOL SPR NAS CT FR VD TRANS X 0,2 ML ...",N6A9 - ANTIDEPRESSIVOS TODOS OS OUTROS,Novo,1754.22,a
...,...,...,...,...,...,...,...,...,...
28914,ÓXIDO DE MAGNÉSIO;SIMETICONA;HIDRÓXIDO DE ALUM...,61.190.096/0001-92,EUROFARMA LABORATÓRIOS S.A.,SIMECO PLUS,120 MG/ML + 60 MG/ML + 7 MG/ML SUS OR CT FR VD...,A2A4 - ANTIÁCIDOS COM ANTIFLATULENTOS OU CARMI...,Específico,8.72,a
28915,ÓXIDO DE ZINCO,30.222.814/0001-31,VITAMEDIC INDUSTRIA FARMACEUTICA LTDA,VITAGLÓS,5500UI/G + 990UI/G + 150MG/G POM CT BG AL X 45G,D2A - EMOLIENTES PROTETORES DERMATOLÓGICOS,Específico,14.52,a
28916,ÓXIDO DE ZINCO,30.222.814/0001-31,VITAMEDIC INDUSTRIA FARMACEUTICA LTDA,VITAGLÓS,5500UI/G + 990UI/G + 150MG/G POM CX 50 BG AL X...,D2A - EMOLIENTES PROTETORES DERMATOLÓGICOS,Específico,557.31,a
28917,ÓXIDO DE ZINCO;RETINOL;COLECALCIFEROL,73.856.593/0001-66,PRATI DONADUZZI & CIA LTDA,PRATIGLÓS,5000 UI/G + 900 UI/G + 150 MG/G POM CX 50 BG A...,D3A9 - TODOS OUTROS PRODUTOS PARA TRATAMENTO D...,Específico,380.47,a


In [158]:
excel_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28919 entries, 0 to 28918
Data columns (total 8 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   SUBSTÂNCIA                           28919 non-null  object 
 1   CNPJ                                 28919 non-null  object 
 2   LABORATÓRIO                          28919 non-null  object 
 3   PRODUTO                              28919 non-null  object 
 4   APRESENTAÇÃO                         28919 non-null  object 
 5   CLASSE TERAPÊUTICA                   28919 non-null  object 
 6   TIPO DE PRODUTO (STATUS DO PRODUTO)  28919 non-null  object 
 7   PF 0%                                28919 non-null  float64
dtypes: float64(1), object(7)
memory usage: 1.8+ MB


In [7]:
excel_df['APRESENTAÇÃO']

0                   20 MG SOL INJ CT 5 AMP VD TRANS X 1 ML
1        (4,0 + 500,0) MG/ML SOL INJ IV/IM CT 3 AMP VD ...
2             10% SOL INJ IV CX 50 FR AMP VD TRANS X 50 ML
3                 100 MG/ML SOL INJ CX 3 AMP VD AMB X 2 ML
4        140 MG/ML SOL SPR NAS CT FR VD TRANS X 0,2 ML ...
                               ...                        
28914    120 MG/ML + 60 MG/ML + 7 MG/ML SUS OR CT FR VD...
28915      5500UI/G + 990UI/G + 150MG/G POM CT BG AL X 45G
28916    5500UI/G + 990UI/G + 150MG/G POM CX 50 BG AL X...
28917    5000 UI/G + 900 UI/G + 150 MG/G POM CX 50 BG A...
28918                1 G + 10 MG COM EFERV CT TB PLAS X 10
Name: APRESENTAÇÃO, Length: 28919, dtype: object

In [83]:
a = excel_df['APRESENTAÇÃO'][41]
pt = r'MG\s\/\sML|ML/ML|MG/ML|MG/G|GV/ML|MEQ/L|G/L|G/G|MG|MCG|G/ML|TRU|MU/ML|UI/ML|U/G|U/U|\sUI\s|\sU\s|$'
print(a, '=>', re.split(pt, a)[0] + re.search(pt, a).group())

2,00 MG + 0,035 MG COM REV CT BL AL PLAS TRANS X 21  => 2,00 MG


In [118]:
a = excel_df['APRESENTAÇÃO'][44]
pt = r'X\s'
print(a)
print(re.split(pt, a)[-1])
# print(re.search(pt, t).group())
# print(re.findall(pt, t))
# print(t, '=>', re.split(pt, t)[0] + re.search(pt, t).group())

2 MG COM REV X 11 + (2 + 1) MG COM REV X 10 CT BL AL PLAS PVC TRANS
10 CT BL AL PLAS PVC TRANS


In [136]:
a = excel_df['APRESENTAÇÃO'][26]
pt = r'POM|CREM'
print(a)
print(re.search(pt, a).group())

15 MG/G CREM CT TB AL X 30 G
CREM


In [161]:
pt1 = r'MG\s\/\sML|ML/ML|MG/ML|MG/G|GV/ML|MEQ/L|G/L|G/G|MG|MCG|G/ML|TRU|MU/ML|UI/ML|U/G|U/U|\sUI\s|\sU\s|$'
pt2 = r'X\s'
pt3 = r'POM|CREM|GEL|MOLE'
pt4 = r'SOL|XPE|SUS|INJ|ML'

dosagem = []
quantidade = []
tipo = []

for a in excel_df['APRESENTAÇÃO']:
  m1 = re.search(pt1, a)
  d = re.split(pt1, a)[0] + m1.group() if m1 else ''

  m2 = re.split(pt2, a)
  q = m2[-1] if m2 else ''

  m3 = re.search(pt3, a)
  m4 = re.search(pt4, a)
  if m3: t = 'Semissólido'
  elif m4: t = 'Líquido'
  else: t = 'Sólido'

  dosagem.append(d)
  quantidade.append(q)
  tipo.append(t)

  # print(a, '=> DOSAGEM:', d, '| QUANTIDADE:', q, '| TIPO:', t)
  
excel_df['DOSAGEM'] = dosagem
excel_df['QUANTIDADE'] = quantidade
excel_df['TIPO'] = tipo

In [163]:
# excel_df.info()
excel_df.head()

,SUBSTÂNCIA,CNPJ,LABORATÓRIO,PRODUTO,APRESENTAÇÃO,CLASSE TERAPÊUTICA,TIPO DE PRODUTO (STATUS DO PRODUTO),PF 0%,DOSAGEM,QUANTIDADE,TIPO
0,BUTILBROMETO DE ESCOPOLAMINA,61.082.426/0002-07,COSMED INDUSTRIA DE COSMETICOS E MEDICAMENTOS ...,BUSCOPAN,20 MG SOL INJ CT 5 AMP VD TRANS X 1 ML,A3A - ANTIESPASMÓDICOS E ANTICOLINÉRGICOS PUROS,Novo,8.88,20 MG,1 ML,Líquido
1,DIPIRONA MONOIDRATADA;BUTILBROMETO DE ESCOPOLA...,61.082.426/0002-07,COSMED INDUSTRIA DE COSMETICOS E MEDICAMENTOS ...,BUSCOPAN COMPOSTO,"(4,0 + 500,0) MG/ML SOL INJ IV/IM CT 3 AMP VD ...",A3D - ASSOCIAÇÕES DE ANTIESPASMÓDICOS COM ANAL...,Novo,12.32,"(4,0 + 500,0) MG/ML",5 ML,Líquido
2,SULFATO DE MAGNÉSIO HEPTAIDRATADO,05.155.425/0001-93,VASCONCELOS INDUSTRIA FARMACEUTICA E COMERCIO ...,SULFATO DE MAGNÉSIO HEPTAHIDRATADO,10% SOL INJ IV CX 50 FR AMP VD TRANS X 50 ML,K4A2 - SOLUÇÕES ELETROLÍTICAS (>20ML E <100ML),Específico,375.00,10% SOL INJ IV CX 50 FR AMP VD TRANS X 50 ML,50 ML,Líquido
3,CIPIONATO DE TESTOSTERONA,00.923.140/0001-31,EMS SIGMA PHARMA LTDA,DEPOSTERON,100 MG/ML SOL INJ CX 3 AMP VD AMB X 2 ML,"G3B - ANDRÓGENOS EXCLUINDO G3E, G3F",Similar,159.70,100 MG/ML,2 ML,Líquido
4,CLORIDRATO DE ESCETAMINA,51.780.468/0001-87,JANSSEN-CILAG FARMACÊUTICA LTDA,SPRAVATO,"140 MG/ML SOL SPR NAS CT FR VD TRANS X 0,2 ML ...",N6A9 - ANTIDEPRESSIVOS TODOS OS OUTROS,Novo,1754.22,140 MG/ML,"0,2 ML + DISP INAL (28 MG)",Líquido
